In [13]:
import numpy as np
import tensorflow as tf
from DQNagent import DQNagent

In [14]:
agent = DQNagent(4,2)

In [15]:
agent.backward_pass(np.array([[1,1,1,1]]),0,1,np.array([[2,2,2,2]]),False)

[[ 0.18853052 -0.52840495]]
1.3582079872488975


In [16]:
agent.forward_pass(np.array([[1,1,1,1]]))

array([[ 0.5086783, -0.512836 ]], dtype=float32)